# Pytorch를 이용한 숫자 구분
MNIST data set을 이용하여 0~9의 이미지를 구분하는 인공지능을 프로그래밍 해보았습니다.

### 라이브러리 import하기

In [1]:
import torch
import torchvision

### 데이터 로딩
학습에 Mini-batch 방식을 이용하며, 그 크기를 1000으로 설정해줍니다.
MINIST로부터 데이터를 다운로드하고 다운로드한 데이터로부터 미니배치만큼 데이터를 로딩합니다.

In [2]:
batch_size = 1000
mnist_train = torchvision.datasets.MNIST(root="MNIST_data/", train=True, transform=torchvision.transforms.ToTensor(), download=True)
mnist_test = torchvision.datasets.MNIST(root="MNIST_data/", train=False, transform=torchvision.transforms.ToTensor(), download=True)
data_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST\raw



### device를 CPU로 지정

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

### 모델 정의
숫자 손글씨 이미지를 입력했을 때, 이미지가 무엇을 나타내는지 학습시키기 위해 모델을 정의합니다.
이때, 이미지 한장은 $28\times28$ 크기이므로 입력 값은 $784$개이고, 출력값은 해당 이미지가 어떤 숫자인지에 대한 0~9까지의 확률값이므로 총 $10$입니다.
따라서 신경망 모델을 아래의 코드와 같이 정의합니다. 입력층과 출력층으로만 구성되는 단순한 모델입니다. 이때, 가중치 외의 편향값도 사용하므로 bias = TRUE로 설정해줍니다.


In [4]:
linear = torch.nn.Linear(784, 10, bias=True).to(device)

### Loss 계산과 weight 조정
각 훈련은 Loss만큼 weight와 bias를 최적의 값으로 보정합니다. Loss function으로는 Cross Entropy를, 출력층에서 사용하는 활성화 함수로는 Softmax를 사용합니다. 이를 ```CrossEntropyLoss()```에서 적용해줍니다.
그리고 training에 대한 Loss를 최소화하기 위한 weight와 bias를찾기 위해서는 Gradient Descent를 사용합니다. 


In [5]:
loss = torch.nn.CrossEntropyLoss().to(device)
SDG = torch.optim.SGD(linear.parameters(), lr=0.1)

1 Epoch(minibatch로 전체 훈련 데이터 처리 단위)에서 몇 번의 미니배치가 반복되는지, 그리고 몇번의 Epoch만큼 훈련할 것인지에 대해 나타냅니다.

In [6]:
total_batch = len(data_loader) # 60 = 60000 / 1000 (total / batch_size)
training_epochs = 10

### Training
```X = X.view(-1, 28 * 28).to(device)```에서는 tensor의 크기를 변경해줍니다.
```Y = Y.to(device)```는 이미지 데이터에 대한 라벨 값으로, One-hot encoding이 아니라 0~9까지의 값으로 이미지에 대한 의미를 나타냅니다.

```
hypothesis = linear(X)
cost = loss(hypothesis, Y)
```
에서는 각각 입력 이미지에 대한 추정값을 얻고, 추정값과 라벨값인 참값 사이에서의 오차값을 계산합니다.
```
SDG.zero_grad()
cost.backward()
SDG.step()
```
에서는 back propagation을 이용해서 weight과 bias를 보정합니다.
마지막으로 ```print…```에서는 1 Epoch마다 Loss가 얼마나 나오는지 확인합니다.


In [7]:
for epoch in range(training_epochs):
    total_cost = 0
    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)
        
        hypothesis = linear(X)
        cost = loss(hypothesis, Y)
        SDG.zero_grad()
        cost.backward()
        SDG.step()
        total_cost += cost 
    avg_cost = total_cost / total_batch
    print("Epoch:", "%03d" % (epoch+1), "cost =", "{:.9f}".format(avg_cost))

Epoch: 001 cost = 1.157598257
Epoch: 002 cost = 0.640864730
Epoch: 003 cost = 0.534744382
Epoch: 004 cost = 0.483918011
Epoch: 005 cost = 0.452709466
Epoch: 006 cost = 0.431154996
Epoch: 007 cost = 0.414994448
Epoch: 008 cost = 0.402435809
Epoch: 009 cost = 0.392230511
Epoch: 010 cost = 0.383769482


### 정확도 테스트
아래의 코드에서는 훈련을 통해 얻어진 가중치 값과 편향값을 테스트 데이터에 적용해 얼마만큼의 정확도가 나오는지 확인하는 코드입니다.


In [8]:
with torch.no_grad():
    X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.targets.to(device)
    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy: ", accuracy.item())

Accuracy:  0.8992999792098999


### 느낀점
수업 시간에 배운 Loss 계산을 통한 weight와 bias update 과정을 직접 프로그래밍해보며 이론적으로 배웠던 것들을 실제로 접해볼 수 있었습니다. 프로그래밍 코드 자체는 인터넷에서 많이 참고하였으나 각 코드의 의미를 직접 고민해보면서 이제까지 배운 내용을 다시 되짚어볼 수도 있어서 좋았습니다.
더군다나 feedforward network, back propagation 등의 경우 여러 어려운 수식들이 많았었는데 pytorch를 이용하면 어려운 수식을 일일히 입력하지 않고 간단한 코드만으로도 이를 구현할 수 있다는 점이 매우 신기했습니다.